<div style="color:red;font-size:40px;">
<center>
<font size="6" >
به نام خدا 
<br><br>
<font size="5" color="grey" >
<b>
Github
سایت
README
بازیابی بر روی فایل‌های 

</b>
<br>
<font size="4" color="black">
دانشجویان:
<br>
 (98101339) سایه جارالهی 
<br>
 (98171104) بردیا محمدی 
<br>
امیررضا سلیمان‌بیگی (98109837)
<br>
استاد: دکتر احسان‌الدین عسگری
</center>
<br>
</div>

## Crawling

First, the url of different repos are crawled. Consequently, for each repo, th README.md is crawled. Also, empty readmes are ignored.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
import re 

def get_readme(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    html_code = soup.find("article")
    sents = []
    for p in html_code.find_all(['td','p', 'li', re.compile('^h\d+$')]):
        text = p.get_text()
        if text and text.replace('\n', ''):
            sents.append(text)
    return sents


def scrape_readmes(url_list):
    readmes = []
    for idx, url in enumerate(url_list):
        print(f"{idx + 1} / {len(url_list)} | Accessing {url}")
        try:
            readme = get_readme(url)
            if len(readme):
                readmes.append({"url": url, "readme_html": readme})
        except:
            print(f"Couldn't access {url}")
        time.sleep(3)  # needed to not overload GitHub -> getting blocked
    readmes_df = pd.DataFrame(readmes)
    return readmes_df

In [ ]:
topics = [
    'game',
    'hardware', 
    'computer+vision', 
    'artificial+intelligence', 
    'application', 
    'ios', 
    'multiplatform', 
    'deep+learning', 
    'cloud', 
    'verilog'
]
valid_topic_urls = [f'https://github.com/search?o=desc&p={i}&q={topic}&s=stars&type=Repositories' for i in range(1, 21) for topic in topics]


In [ ]:
def get_repo_urls(url):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    repos = soup.find_all('a', class_='v-align-middle')
    time.sleep(3) # needed to not overload GitHub -> getting blocked
    return ['https://github.com'+repo.get('href') for repo in repos]

def scrape_urls():
    repo_urls = []
    for i, url in enumerate(valid_topic_urls):
        print('scraping url', i)
        if url not in repo_urls:
            repos = get_repo_urls(url)
            repo_urls.extend(repos)
    return repo_urls
    

In [ ]:
repo_urls = scrape_urls()

In [ ]:
f = open('urls.json')
repo_urls = json.load(f)
readmes_df = scrape_readmes(repo_urls)
readmes_df.to_json("readmes.json", orient="records")

##Load data from files

In [ ]:
readmes_df = pd.read_json('readmes.json', orient= 'records')
tokens_df = pd.read_json('token.json', orient= 'records')
paragraph_df = pd.read_json('string.json', orient='records')

## Preprocess(Normalization, Tokenizarion, Removing Stopwords)

In [ ]:
import nltk
import string
import json
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from pathlib import Path

lemmatizer = WordNetLemmatizer()
stemmer  = PorterStemmer()

def remove_emoji(string):
    b'\\U0001f923'
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return emoj.sub(r'', string)

def normalize_tokens(tokens, remove_stopword, stopwords_domain=['https']):
    
    # Remove stopwords
    if remove_stopword:
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_tokens = [word for word in tokens if (word.lower() not in stopwords_domain + stopwords)]
    else:
        normalized_tokens = tokens
        
    # remove links
    normalized_tokens = [word for word in normalized_tokens if not re.match(r'^//.*(/?)$', word)]
    
    # Remove punctuations
    normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]
    
    # Convert everything to lowercase
    normalized_tokens = [word.lower() for word in normalized_tokens]

    normalized_tokens = [word for word in normalized_tokens if len(word)>1]
    
    
    # Convert numbers to "Number" token
    normalized_tokens = [word if not re.match(r'(^| )\d+(\.\d+)*($| )', word) else 'NUMBER' for word in normalized_tokens]
        
    return normalized_tokens

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
    
def non_english(text):
    t_list = word_tokenize(text)
    res = []
    for i in t_list:
        if isEnglish(i):
            res.append(i)
    return ' '.join(res)



def preprocess(text, lemmatize=True, remove_stopword=True, token=True, re_sub_domain='\'|`|-'):

    # remove non enlish
    text = non_english(text)

    # replace sub_domain with space
    text = re.sub(re_sub_domain, ' ', text)

    # remove emoji
    text = remove_emoji(text)

    tokens = word_tokenize(text)
    
    # normalization
    tokens = normalize_tokens(tokens, remove_stopword)
    
    # lemmatization
    if lemmatize:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    if token:
        return tokens
    else:
        return ' '.join(tokens)


def get_preprocessed_token(file_path='readmes.json'):
    data = json.loads(Path(file_path).read_text())
    docs = []
    for row in data:
        text = row['readme_html']
        preprocessed_tokens = []
        for paragraph in text:
            preprocessed_tokens += preprocess(paragraph)
        docs.append({'url': row['url'], 'text': preprocessed_tokens})
    Path('token.json').write_text(json.dumps(docs))


def get_preprocessed_str(file_path='readmes.json'):
    data = json.loads(Path(file_path).read_text())
    docs = []
    for row in data:
        text = row['readme_html']
        preprocessed_paragraphs = []
        for paragraph in text:
            preprocessed_paragraphs.append(preprocess(paragraph, lemmatize=False, remove_stopword=False, token=False))
        docs.append({'url': row['url'], 'text': preprocessed_paragraphs})
    Path('string.json').write_text(json.dumps(docs))

In [ ]:
get_preprocessed_token()

In [ ]:
get_preprocessed_str()

## Boolean Retrival
In this section, boolean retrival is implemented. The format of input query should be: 

word1 AND word2 word3 OR word4 word5 NOT word6

The matrix is implemented using inverted index. 


In [ ]:
import json 
from copy import deepcopy
class BooleanRetrival:

    def __init__(self, docs, k = 10):
        self.inverted_index = {}
        self.docs = docs
        self.query_keywords = ['NOT', 'ORNOT', 'OR', 'AND']
        self.k = k 
    
    def create_inverted_index(self):
        for i, row in self.docs.iterrows():
            tokens = row['text']
            url = row['url']
            for word in tokens:
                if word in self.inverted_index.keys():
                    self.inverted_index[word].add(url)
                else:
                    self.inverted_index.update({word:{url}})
    
    def dump_dict_json(self):
        with open('inverted_index.json', 'w') as f:
            f.write(json.dumps(self.inverted_index))

    def add_and_to_query(self, query):
        temp_query = deepcopy(query)
        changed_count = 0 
        for i in range(len(query)-1):
            if query[i] not in self.query_keywords and query[i+1] not in self.query_keywords:
                temp_query.insert(i+1 + changed_count,'AND')
                changed_count +=1 
        return temp_query
    
    def normalize_query(self, query):
        result = []
        for text in query:
            if text not in self.query_keywords:
                result.append(preprocess(text, token = False))
            else:
                result.append(text)
        return result
        
    def tokenize_query(self, query):
        query = query.replace('OR NOT', 'ORNOT')
        query = query.split(' ')
        query = self.add_and_to_query(query)
        query = self.normalize_query(query)
        return query
    
    def process_query(self, query):
        query = self.tokenize_query(query)
        if not query[0] in self.inverted_index.keys():
            result = set(self.docs.loc[:,"url"])
        else:
            result = set(self.inverted_index[query[0]])
        i = 1
        while i < len(query):
            result = self.handle_operation(result,query[i+1], query[i])
            i += 2
        res = []
        for i in result:
            res.append({'score':1.0, 'url': i})
        return res[0:self.k]
    
    def handle_operation(self, result:set, second_word:str, operation:str):
        if not second_word in self.inverted_index.keys():
            return result
        second_list = self.inverted_index[second_word]
        if operation == 'AND':
            return result.intersection(second_list)
        elif operation=='OR':
            return result.union(second_list)
        elif operation == 'ORNOT' :
            all_urls = self.docs.loc[:,"url"]
            return result.union((set(tokens_df.loc[:,'url'])) - self.inverted_index[second_word])
        elif operation == 'NOT':
            return result - second_list

In [ ]:
model = BooleanRetrival(tokens_df)
model.create_inverted_index()


In [ ]:
result = model.process_query('Hardware cores OR Arduino AND API')
result

[{'score': 1.0, 'url': 'https://github.com/Gamua/Starling-Framework'},
 {'score': 1.0, 'url': 'https://github.com/vsouza/awesome-ios'},
 {'score': 1.0, 'url': 'https://github.com/gbdev/awesome-gbdev'},
 {'score': 1.0, 'url': 'https://github.com/nolimits4web/swiper'},
 {'score': 1.0, 'url': 'https://github.com/pikvm/pikvm'},
 {'score': 1.0, 'url': 'https://github.com/hybridgroup/gobot'},
 {'score': 1.0, 'url': 'https://github.com/microsoft/CNTK'},
 {'score': 1.0, 'url': 'https://github.com/ory/hydra'},
 {'score': 1.0, 'url': 'https://github.com/WICG/webusb'},
 {'score': 1.0,
  'url': 'https://github.com/mahmoud/awesome-python-applications'}]

## Shunt algorithm: 
This can be used if our queries have paranthesis or they are inside each other. As was mentioned in quera, there is no need for the boolean retrival module to handle queries with paranthesis. However, we implemented that feature. 

In [ ]:
from collections import namedtuple

opinfo = namedtuple('Operator', 'precedence associativity')
operator_info = {
    "AND": opinfo(0, "L"),
    "OR": opinfo(0, "L"),
    "NOT": opinfo(0, "L"),
    "ORNOT": opinfo(0,"L")
}

def shunt(tokens):
    tokens += ['end']
    operators = []
    output = []
    while len(tokens) != 1:
        current_token = tokens.pop(0)

        if current_token not in operator_info.keys() and current_token not in ['(', ')']:
            output.append(current_token)
        elif current_token in operator_info.keys():
            while len(operators):
                satisfied = False
                if operators[-1] not in ["(", ")"]:
                    satisfied = True
                satisfied = satisfied and operators[-1] != "("
                if not satisfied:
                    break
                output.append(operators.pop())
            operators.append(current_token)
        elif current_token == "(":
            operators.append(current_token)

        elif current_token == ")":
            while len(operators) and operators[-1] != '(':
                output.append(operators.pop())
            if len(operators) != 0 and operators[-1] == "(":
                operators.pop()
    output.extend(operators[::-1])
    return output



## Tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
from pathlib import Path

class Tfidf:
    
    def __init__(self, file_name='token.json'):
        data = json.loads(Path(file_name).read_text())
        corpus = []
        self.urls = []
        for row in data:
            corpus.append(' '.join(row['text']))
            self.urls.append(row['url'])

        self.vectorizer = TfidfVectorizer(
            input='content', 
            lowercase=False, 
            analyzer='word', 
            ngram_range=(1, 1), 
            max_df=0.90, 
            min_df=1, 
            max_features=None, 
            vocabulary=None, 
            binary=False, 
            norm='l2', 
            use_idf=True, 
            smooth_idf=True, 
            sublinear_tf=True
        )
        
        self.X = self.vectorizer.fit_transform(corpus)
        
    def process_query(self, query, url_num=10):
        
        query = preprocess(query)
        words = self.vectorizer.get_feature_names()
        score = np.zeros(self.X.shape[0])
        for word in query:
            if word in words:
                index = words.index(word)
                score += self.X[:, index].toarray()[:, 0]
        max_scores_docs = list(np.argsort(score))
        
        related_urls = []
        scores = []
        for doc in reversed(max_scores_docs):
            if self.urls[doc] not in related_urls:
                related_urls.append(self.urls[doc])
                scores.append(score[doc])
            if len(related_urls) == url_num:
                break
            
        return related_urls, scores

    def print_results(self, query):
        for url, score in zip(*self.process_query(query)):
            print(f'url: {url}, score: {score:.5f}')

In [ ]:
model = Tfidf()
model.print_results(query='face detection')

url: https://github.com/cvzone/cvzone, score: 0.40511
url: https://github.com/arunponnusamy/cvlib, score: 0.21360
url: https://github.com/eduardolundgren/tracking.js, score: 0.20274
url: https://github.com/bnosac/image, score: 0.19961
url: https://github.com/chandrikadeb7/Face-Mask-Detection, score: 0.19888
url: https://github.com/mpatacchiola/deepgaze, score: 0.19452
url: https://github.com/php-opencv/php-opencv-examples, score: 0.19303
url: https://github.com/symisc/sod, score: 0.19126
url: https://github.com/extreme-assistant/survey-computer-vision-2020, score: 0.17466
url: https://github.com/amusi/awesome-ai-awesomeness, score: 0.16635


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## FastText

In [ ]:
! pip install engines
from gensim.models.fasttext import FastText
import numpy as np
import pandas as pd
import scipy

class FastTextModel:

    def __init__(self, data, train=False, k=10):
        self.data = data
        if train:
            self.ft_model = self.train()
        else:
            self.ft_model = FastText.load('_fasttext.model')
        self.data_embedding = self.get_data_embedding_avg()
        self.k = k

    def get_words(self, doc):
        words = []
        for sent in doc:
            words.extend(sent.split(' '))
        return words

    def get_tok_text(self):
        tok_text = []
        for doc in self.data['readme_html']:
            tok_text.append(self.get_words(doc))
        return tok_text

    def train(self):
        ft_model = FastText(sg=1, size=100, window=10, min_count=2, negative=15, min_n=2, max_n=5)
        tok_text = list(self.data.loc[:, 'text'])
        ft_model.build_vocab(tok_text)
        ft_model.train(
            tok_text,
            epochs=6,
            total_examples=ft_model.corpus_count,
            total_words=ft_model.corpus_total_words)
        ft_model.save('_fasttext.model')  # save
        return ft_model

    def get_data_embedding_avg(self) -> dict:
        '''

        :return: a dict: the keys are the urls of data and the value is the embedding average
        '''
        docs_avg = dict()
        for index, row in self.data.iterrows():
            words = row['text']
            url = row['url']
            docs_avg[url] = np.mean([self.ft_model.wv[word] for word in words], axis=0)
        return docs_avg

    def get_query_embedding(self, query_tokens):
        return np.mean([self.ft_model.wv[word] for word in query_tokens], axis=0)

    def tokenize_query(self, query: str):
        query = preprocess(query, token=True)
        return query

    def search_query(self, query):
        tokens = self.tokenize_query(query)
        q_embedding = self.get_query_embedding(tokens)
        cosine_sim_values = {}
        for url, embedding_value in self.data_embedding.items():
            cosine_sim_values.update({url: self.cosine_sim(embedding_value, q_embedding)})
        return self.calculate_best_k(cosine_sim_values)

    def calculate_best_k(self, cosine_sim_values:dict):
        res = sorted(cosine_sim_values.items(), key=lambda x: x[1], reverse = True) [0: self.k]
        return res

    def cosine_sim(self, x, y):
        return 1- scipy.spatial.distance.cosine(x, y)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 8.2 MB/s 
     |████████████████████████████████| 145 kB 61.1 MB/s 
     |████████████████████████████████| 178 kB 87.7 MB/s 
     |████████████████████████████████| 61 kB 9.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: chardet
    Found existing installation: chardet 3.0.4
    Uninstalling chardet-3.0.4:
      Successfully uninstalled chardet-3.0.4
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.5.18.1
    Uninstalling certifi-2022.5.18.1:
      Successfully uninstalled certifi-2022.5.18.1
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's depende

## Initialize FastText model 

In [ ]:
# by setting train value to True, you have to wait about 8 minutes for the training process
ft_model = FastTextModel(tokens_df, train=False, k=10)
a = ft_model.search_query('atari game')

In [ ]:
a = ft_model.search_query('internet of things circuit')
a

[('https://github.com/dawsonjon/Chips-2.0', 0.8632255792617798),
 ('https://github.com/itsFrank/MinecraftHDL', 0.8601580262184143),
 ('https://github.com/aappleby/MetroBoy', 0.8593295216560364),
 ('https://github.com/yadox666/The-Hackers-Hardware-Toolkit',
  0.8564713597297668),
 ('https://github.com/stevehoover/warp-v', 0.8494113683700562),
 ('https://github.com/pervognsen/bitwise', 0.8475880026817322),
 ('https://github.com/GlasgowEmbedded/glasgow', 0.8472965359687805),
 ('https://github.com/fossasia/neurolab-hardware', 0.8455127477645874),
 ('https://github.com/ZipCPU/zipcpu', 0.8447715044021606),
 ('https://github.com/ZipCPU/autofpga', 0.8443008065223694)]

## Initialize BERT model 

We will use the distilbert-base-nli-stsb-mean-tokens model.

Faiss is a library for efficient similarity search and clustering of dense vectors.


In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers


import numpy as np
import pandas as pd
import faiss

import torch
from sentence_transformers import SentenceTransformer
import scipy 
from operator import itemgetter

class TransformerModel:

    def __init__(self, data):
        
        self.model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
        if torch.cuda.is_available():
            model = self.model.to(torch.device("cuda"))
        self.data = data
        self.list_to_string()
        self.set_id()
        self.embedding = self.get_data_embedding()
        self.index = self.set_indexes()

    def list_to_string(self):
      #Change list of doc string to string
        for i in range(0, len(self.data['text'])):
          self.data['text'][i] = ' '.join(map(str, self.data['text'][i]))

    def get_data_embedding(self):
        return self.model.encode(self.data['text'].tolist(), show_progress_bar=True)

    def vector_search(self, query, model, index, num_results=10):
        vector = model.encode(list(query))
        D, I = index.search(np.array(vector).astype("float32"), k=num_results)
        return D, I

    def set_id(self):
        self.data['id'] = range(0, len(self.data))
    
    def set_indexes(self):
        embeddings = np.array([embedding for embedding in self.embedding]).astype("float32")
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index = faiss.IndexIDMap(index)
        index.add_with_ids(embeddings, self.data.id.values)
        return index

    def get_recommendation(self, user_query):
        D, I = self.vector_search([user_query], self.model, self.index, num_results=10)
        print(f'L2 distance: {D.flatten().tolist()}\n\nReadme IDs: {I.flatten().tolist()}')
        return [self.data['url'][i] for i in I.flatten().tolist()]

    def get_recommendation2(self, query):
        queries = [query]
        query_embeddings = self.model.encode(queries)
        number_top_matches = 10
        for query, query_embedding in zip(queries, query_embeddings):
            distances = scipy.spatial.distance.cdist(query_embeddings, self.embedding, "cosine")[0]
            results = zip(range(len(distances)), distances)
            results = sorted(results, key=lambda x: x[1])
            temp = []
            for idx, distance in results[0:number_top_matches]:
              temp.append(str(1-distance) + ' ' + readmes_df['url'][idx])
            return temp

In [ ]:
t_model = TransformerModel(readmes_df)
a = t_model.get_recommendation('password manager')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

L2 distance: [155.1759796142578, 194.96578979492188, 251.8955841064453, 255.93487548828125, 262.82568359375, 263.5928039550781, 264.57244873046875, 267.7922668457031, 268.36004638671875, 271.2803039550781]

Readme IDs: [355, 1153, 1249, 1385, 570, 1191, 840, 402, 577, 565]


In [ ]:
a

['https://github.com/marcwebbie/passpie',
 'https://github.com/keepassxreboot/keepassxc',
 'https://github.com/keycloak/keycloak',
 'https://github.com/houtianze/bypy',
 'https://github.com/jpillora/cloud-torrent',
 'https://github.com/owncloud/core',
 'https://github.com/kzlekk/HWSensors',
 'https://github.com/Gazler/githug',
 'https://github.com/facebookarchive/RakNet',
 'https://github.com/GoogleCloudPlatform/python-docs-samples']

## Report

<div dir="rtl">
<font size=5>
<b>
گزارش انجام پروژه:
</b>
<font size =3>
<br>
در این تمرین، فایل‌های README گیت‌هاب با استفاده از crawling جمع آوری شده و پس از انجام پیش پردازش بر روی آن‎‌ها، چهار مدل مختلف برای بازیابی اطلاعات و جستجوی مطالب قرار گرفته است. 
<br>
<b>Crawling:</b><br>
در گام اول Crawling داده انجام شده است. با توجه به اینکه در گیت‌هاب ریپازیتوری های مختلفی با موضوعات بسیار گوناگون وجود دارد، با جمع آوری داده در اندازه محدود‌ (حدود ۱۶۰۰ ریپازیتوری)‌ نمیتوان به طور مناسب جستجو را انجام داد. به این جهت، کرال کردن داده ها بر روی ۱۰ دسته کلی از ریپازیتوری‌ها (بازی، سخت افزار، بینایی ماشین، هوش مصنوعی، اپلیکیشن، ios، مولتی پلتفرم، دیپ لرنینگ، فضای ابری، وریلاگ) انجام شده است. از هریک از این ریپازیتوری ها 20 صفحه ( که هر یک شامل 10 ریپازیتوری است) ذخیره شده و README آنها مورد بررسی قرار گرفته است. 
<br>
<b>PreProcessing:</b>
<br>
چالش بعدی در این زمینه، Preprocessing بر روی داده ها برای رسیدن به داده مطلوب بوده است. در این بخش، در صورتی که داده ها به زبانی به جز انگلیسی باشند (موردی که مشاهده شد آن است که داده هایی به زبان چینی و هندی بسیار رایج است) آن داده ها را حذف میکند. همچنین از lemmatizer استفاده میشود تا کلمات همخانواده به یک کلمه یکسان بروند. در نهایت نیز از تمام اعداد با عبارت NUMBER جانشین شده، از Tokenizerو Normalizer و Stopword Removal استفاده شده است تا خروجی مناسب تولید شود. در برخی از الگوریتم های بازیابی از توکن ها و در برخی از متن کامل نورمالایزر شده استفاده میشود. همچنین در بسیاری از repository ها لینک‌هایی وجود دارد که آنها را نیز حذف می‌کنیم. در صورتی طول کلمه کمتر از 2 باشد آن را حذف می‌کنیم. همچنین emoji های موجود را نیز حذف می‌کنیم.
<br>

</div>


<div dir= "rtl">
<font size =3>
<b>Boolean Retrival:</b>
<br>
در گام اول جستجو Boolean انجام می‌شود. این جستجو تنها به حضور کلمات در متن‌ها توجه دارد. در نتیجه هرچه تعداد کلمات کوئری‌ها بیشتر باشد و این کلمات به هم نامرتبط باشد،‌ تعداد خروجی‌ها نیز کمتر می‌شود. برای استفاده از این الگوریتم، ابتدا inverted matrix از روی داده‌ها به دست آورده میشود و در بخش بعدی، پاسخ دادن به کوئری‌ها انجام می‌شود. فرمت کوئری‌ها به صورتی است که تعدادی کلمه است که با AND, OR, NOT, OR NOT جدا شده اند. درصورتی‌که بین دو کلمه هیچ عبارت متصل کننده‌ای نباشد نیز AND در نظر گرفته میشود. 
مقدار mrr عدد 0.61 است

</div>

In [ ]:
result = model.process_query('atari game NOT deep')
result

[{'score': 1.0, 'url': 'https://github.com/dkhamsing/open-source-ios-apps'},
 {'score': 1.0, 'url': 'https://github.com/evilsocket/pwnagotchi'},
 {'score': 1.0, 'url': 'https://github.com/leereilly/games'},
 {'score': 1.0, 'url': 'https://github.com/microsoft/malmo'},
 {'score': 1.0, 'url': 'https://github.com/gbdev/awesome-gbdev'},
 {'score': 1.0, 'url': 'https://github.com/OpenEmu/OpenEmu'},
 {'score': 1.0, 'url': 'https://github.com/charcole/Z3'}]

In [ ]:
result = model.process_query('cloud service OR IoT AND docker')
result

[{'score': 1.0, 'url': 'https://github.com/dkhamsing/open-source-ios-apps'},
 {'score': 1.0, 'url': 'https://github.com/uraimo/SwiftyGPIO'},
 {'score': 1.0, 'url': 'https://github.com/onceupon/Bash-Oneliner'},
 {'score': 1.0,
  'url': 'https://github.com/alirezadir/Production-Level-Deep-Learning'},
 {'score': 1.0, 'url': 'https://github.com/anaibol/awesome-serverless'},
 {'score': 1.0, 'url': 'https://github.com/Countly/countly-server'},
 {'score': 1.0, 'url': 'https://github.com/hortonworks/cloudbreak'},
 {'score': 1.0, 'url': 'https://github.com/Nepxion/Discovery'},
 {'score': 1.0, 'url': 'https://github.com/sqshq/piggymetrics'},
 {'score': 1.0, 'url': 'https://github.com/netdata/netdata'}]

In [ ]:
result = model.process_query('hybrid transactional processing OR recommender OR system')
result

[{'score': 1.0, 'url': 'https://github.com/mgba-emu/mgba'},
 {'score': 1.0, 'url': 'https://github.com/vlang/ved'},
 {'score': 1.0, 'url': 'https://github.com/udacity/deep-learning-v2-pytorch'},
 {'score': 1.0, 'url': 'https://github.com/SFTtech/openage'},
 {'score': 1.0, 'url': 'https://github.com/mfontanini/libtins'},
 {'score': 1.0, 'url': 'https://github.com/theseus-os/Theseus'},
 {'score': 1.0,
  'url': 'https://github.com/playgameservices/play-games-plugin-for-unity'},
 {'score': 1.0, 'url': 'https://github.com/amzeratul/halley'},
 {'score': 1.0, 'url': 'https://github.com/mtmd/FPGA_Based_CNN'},
 {'score': 1.0, 'url': 'https://github.com/GuitarML/SmartGuitarAmp'}]

In [ ]:
result = model.process_query('Kubernetes OR docker AND tutorial')
result

[{'score': 1.0, 'url': 'https://github.com/dkhamsing/open-source-ios-apps'},
 {'score': 1.0, 'url': 'https://github.com/uraimo/SwiftyGPIO'},
 {'score': 1.0, 'url': 'https://github.com/onceupon/Bash-Oneliner'},
 {'score': 1.0, 'url': 'https://github.com/supervisely/supervisely'},
 {'score': 1.0, 'url': 'https://github.com/gravitl/netmaker'},
 {'score': 1.0, 'url': 'https://github.com/anaibol/awesome-serverless'},
 {'score': 1.0,
  'url': 'https://github.com/goodrahstar/my-awesome-AI-bookmarks'},
 {'score': 1.0, 'url': 'https://github.com/sqshq/piggymetrics'},
 {'score': 1.0, 'url': 'https://github.com/drakkan/sftpgo'},
 {'score': 1.0, 'url': 'https://github.com/chainer/chainer'}]

In [ ]:
result = model.process_query('accelerate deep learning applications AND GPU OR CPU')
result

[{'score': 1.0, 'url': 'https://github.com/Tessellate-Imaging/monk_v1'},
 {'score': 1.0, 'url': 'https://github.com/onceupon/Bash-Oneliner'},
 {'score': 1.0, 'url': 'https://github.com/hortonworks/cloudbreak'},
 {'score': 1.0, 'url': 'https://github.com/Tencent/ncnn'},
 {'score': 1.0, 'url': 'https://github.com/jmahler/mips-cpu'},
 {'score': 1.0, 'url': 'https://github.com/alibaba/MNN'},
 {'score': 1.0,
  'url': 'https://github.com/PacktPublishing/Hands-On-GPU-Accelerated-Computer-Vision-with-OpenCV-and-CUDA'},
 {'score': 1.0, 'url': 'https://github.com/keroro824/HashingDeepLearning'},
 {'score': 1.0, 'url': 'https://github.com/openrisc/mor1kx'},
 {'score': 1.0, 'url': 'https://github.com/ultraembedded/biriscv'}]

In [ ]:
result = model.process_query('React web application development NOT webStorm')
result


[{'score': 1.0, 'url': 'https://github.com/plausible/analytics'},
 {'score': 1.0, 'url': 'https://github.com/ory/hydra'},
 {'score': 1.0, 'url': 'https://github.com/anaibol/awesome-serverless'},
 {'score': 1.0,
  'url': 'https://github.com/serhii-londar/open-source-mac-os-apps'},
 {'score': 1.0, 'url': 'https://github.com/ellisonleao/magictools'},
 {'score': 1.0, 'url': 'https://github.com/aws-amplify/amplify-js'},
 {'score': 1.0, 'url': 'https://github.com/donnemartin/awesome-aws'},
 {'score': 1.0, 'url': 'https://github.com/vsouza/awesome-ios'},
 {'score': 1.0, 'url': 'https://github.com/sergiocasero/votlin-app'},
 {'score': 1.0, 'url': 'https://github.com/serverless/examples'}]

<div dir = "rtl">
<font size=3>
<b>Shunt Algorithm for Boolean Retrival:</b> 
</div>

<div dir="rtl">
الگوریتم Shunt یک الگوریتم برای تبدیل یک عبارت infix به postfix است. درصورتی که کوئری وارد شده دارای اولویت و پرانتزگذاری باشد، ابتدا این الگوریتم را روی کوئری صدا میزنیم. پس از آنکه کوئری به حالت postfix در بیاید میتوان با اجرای کوئری و استفاده از استک برای نگه داری بخش های مختلف کوئری،‌ جواب را پیدا کرد. 
</div>

<div dir = "rtl">
<font size=3>
<b>Tf-idf:</b> 
</div>

<div dir="rtl">
از کلاس TfidfVectorizer در ماژول sklearn.feature_extraction.text استفاده می‌کنیم. 'input='content را استفاده می‌کنیم که ورودی را string در نظر بگیرد، 'analyzer='word استفاده می‌کنیم تا پردازش در سطح کلمه باشد. از مدل unigram نیز استفاده می‌کنیم. کلماتی که در بیشتر از 0.9 داکیومنت‌ها هستند را نادیده می‌گیریم. از 'L2 norm' برای normalize هر سطر استفاده می‌کنیم. همچنین به جای tf از 1 + log(tf) برای به جای tf در رابطه tf-idf استفاده می‌کنیم. از smoothing نیز استفاده می کنیم به طوری که یک را با فرکانس داکیومنت‌ها جمع می‌کنیم تا از تقسیم شدن به صفر جلوگیری کند. MRR در این بخش برابر با 0.82 است.
</div>

In [ ]:
model = Tfidf()
model.print_results(query='applications of deep learning')

url: https://github.com/amusi/Deep-Learning-Interview-Book, score: 0.54790
url: https://github.com/thibo73800/aihub, score: 0.53130
url: https://github.com/ankonzoid/artificio, score: 0.38436
url: https://github.com/tirthajyoti/Papers-Literature-ML-DL-RL-AI, score: 0.35261
url: https://github.com/ashishpatel26/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code, score: 0.32714
url: https://github.com/rasmusbergpalm/DeepLearnToolbox, score: 0.31214
url: https://github.com/PacktPublishing/Artificial-Intelligence-with-Python, score: 0.31167
url: https://github.com/PacktPublishing/PyTorch-Computer-Vision-Cookbook, score: 0.30757
url: https://github.com/khanhnamle1994/computer-vision, score: 0.30675
url: https://github.com/openai/spinningup, score: 0.30622


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
model = Tfidf()
model.print_results(query='Game studio projects')

url: https://github.com/chrismaltby/gb-studio, score: 0.40688
url: https://github.com/WillKoehrsen/ai-projects, score: 0.39587
url: https://github.com/cping/LGame, score: 0.24588
url: https://github.com/UnityTechnologies/open-project-1, score: 0.23797
url: https://github.com/watabou/pixel-dungeon, score: 0.23280
url: https://github.com/FlaxEngine/FlaxEngine, score: 0.22696
url: https://github.com/cocos2d/cocos2d-x, score: 0.22609
url: https://github.com/stride3d/stride, score: 0.22274
url: https://github.com/GameServerManagers/LinuxGSM, score: 0.21907
url: https://github.com/ppy/osu, score: 0.21337


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
model = Tfidf()
model.print_results(query='visual recognition and image segmentation tasks')

url: https://github.com/khanhnamle1994/computer-vision, score: 0.47617
url: https://github.com/yassouali/ML-paper-notes, score: 0.40179
url: https://github.com/hoya012/CVPR-2019-Paper-Statistics, score: 0.37243
url: https://github.com/ArcherFMY/Paper_Reading_List, score: 0.33788
url: https://github.com/bnosac/image, score: 0.33496
url: https://github.com/amusi/awesome-ai-awesomeness, score: 0.32714
url: https://github.com/google-research/scenic, score: 0.31677
url: https://github.com/microsoft/computervision-recipes, score: 0.30837
url: https://github.com/balavenkatesh3322/CV-pretrained-model, score: 0.30330
url: https://github.com/SnailTyan/deep-learning-papers-translation, score: 0.29992


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<div dir = "rtl">
<font size=3>
<b>FastText:</b> 
</div>

<div dir = "rtl">
در این روش، در ابتدا مدل را با استفاده از توکن‌های مربوط به README ها ترین میکنیم. همچنین امبدینگ مربوط به هر داک را محاسبه میکنیم. پس از آن کوئری را دریافت کرده و با همان مراحلی که داک‌ها را توکنایز کرده بودیم،‌این کوئری را هم توکنایز و نرمالایز می کنیم. هدف آن است که متن و کوئری در یک فضای کلمات قرار گیرند. در گام بعدی بردار امبدینگ مربوط به کوئری را هم حساب میکنیم. در انتها برای آنکه مشخص شود فاصله بین کدام دو بردار امبدینگ (کوئری و داک ها) از همه کمتر است،‌ فاصله کسینوسی را حساب میکنیم. هرچقدر این مقدار کمتر باشد، دو بردار نزدیکتر بوده و در نتیجه داک در رتبه بالاتری است. البته قابل توجه است که مدل های آماده pretrained برا ترین کردن اولیه بهتر از ترین کردن با دیتا به صورت unsupervised است. اما چون کوئری نیز از همان فضای کلمات متن ها آمده است، در نتیجه مشکل زیادی پیش نمی آید. 
مقدار mrr عدد 0.81 است.

</div>

In [ ]:
a = ft_model.search_query('deep learning tutorial')
a

[('https://github.com/amusi/Deep-Learning-Interview-Book', 0.9012079834938049),
 ('https://github.com/lexfridman/mit-deep-learning', 0.8855657577514648),
 ('https://github.com/nivu/ai_all_resources', 0.8838083148002625),
 ('https://github.com/goodrahstar/my-awesome-AI-bookmarks', 0.882518470287323),
 ('https://github.com/mbadry1/DeepLearning.ai-Summary', 0.8814557194709778),
 ('https://github.com/thibo73800/aihub', 0.880732536315918),
 ('https://github.com/khanhnamle1994/computer-vision', 0.8795418739318848),
 ('https://github.com/ujjwalkarn/Machine-Learning-Tutorials',
  0.871427595615387),
 ('https://github.com/rasmusbergpalm/DeepLearnToolbox', 0.8713003993034363),
 ('https://github.com/iamtrask/Grokking-Deep-Learning', 0.8703610301017761)]

In [ ]:
a = ft_model.search_query('deploy and maintanance')
a

[('https://github.com/Netflix/chaosmonkey', 0.857405960559845),
 ('https://github.com/hashicorp/nomad', 0.8478212356567383),
 ('https://github.com/vercel/vercel', 0.8472731113433838),
 ('https://github.com/spinnaker/spinnaker', 0.8463647365570068),
 ('https://github.com/anaibol/awesome-serverless', 0.840377926826477),
 ('https://github.com/BretFisher/udemy-docker-mastery', 0.8396722674369812),
 ('https://github.com/googleforgames/agones', 0.8391356468200684),
 ('https://github.com/aws/aws-cdk', 0.8369144797325134),
 ('https://github.com/okfn-brasil/serenata-de-amor', 0.8337067365646362),
 ('https://github.com/kubesphere/kubesphere', 0.8324718475341797)]

In [ ]:
a = ft_model.search_query('ios with swift')
a

[('https://github.com/Aufree/trip-to-iOS', 0.8696250319480896),
 ('https://github.com/jordansinger/messages-multiplatform-swiftui-sample',
  0.8445234298706055),
 ('https://github.com/dkhamsing/open-source-ios-apps', 0.7529419660568237),
 ('https://github.com/ibireme/YYKit', 0.7346476316452026),
 ('https://github.com/draveness/analyze', 0.724860668182373),
 ('https://github.com/didi/DoKit', 0.7108945846557617),
 ('https://github.com/Tim9Liu9/TimLiu-iOS', 0.7099772691726685),
 ('https://github.com/vsouza/awesome-ios', 0.6968284845352173),
 ('https://github.com/h2y/Shadowrocket-ADBlock-Rules', 0.6928650736808777),
 ('https://github.com/SnapKit/SnapKit', 0.6798034310340881)]

<div dir = "rtl">
<font size=3>
<b>Transformer:</b> 
</div>

<div dir = "rtl">
​​ما از مدل distilbert-base-nli-stsb-mean-tokens استفاده خواهیم کرد که بهترین عملکرد را در تسک های Textual Similarity  در بین نسخه های DistilBERT دارد. علاوه بر این اگرچه این مدل کمی بدتر از BERT است، اما به دلیل داشتن اندازه کوچکتر، بسیار سریعتر است.
 
Faiss  کتابخانه ای برای جستجوی شباهت و خوشه بندی بردارهای دنس است. این شامل الگوریتم‌هایی است که در مجموعه‌هایی از بردارها با هر اندازه حتی آنهایی که در RAM جا نمی‌شوند جستجو می ‌کنند.
 
این یک index ساده که یک جست و جو با فاصله‌ی L2 brute-force که به صورت خطی افزایش پیدا میکند انجام میدهد.
برای ایجاد یک index با بردارهای بدنه readme  این کار را انجام می دهیم:
نوع داده بردارهای readme را به float32 تغییر میدهیم.یک index میسازیم و به اندازه بعد بردارهایی که روی آنها عملیات انجام خواهد شد.ایندکس را به IndexIDMap ارسال میکنیم و آبجکتی که  شیئی که به ما امکان می‌دهد یک لیست دلخواه از شناسه‌ها برای بردارهای ایندکس شده ارائه کنیم.بردارهای readme و نقشه شناسه آنها را به ایندکس اضافه میکنیم که بردارها را به شناسه readme آنها مپ میکنیم.
 
ایندکسی که ما ساختیم یک جستجوی KNN را انجام می دهد و در آخر باید تعداد همسایه هایی که باید برگردانده شوند را نشان میدهیم.
 مقدار mrr برابر با 0.86 است. 
</div>

q1 = "deep learning papers"

```
L2 distance: [205.97442626953125, 210.46890258789062, 213.1353302001953, 213.1353302001953, 214.97361755371094, 218.3271484375, 220.17562866210938, 223.4440460205078, 223.4635772705078, 223.55410766601562]

Readme IDs: [467, 74, 23, 464, 1377, 1082, 270, 508, 172, 1285]
['https://github.com/tflearn/tflearn',
 'https://github.com/floodsung/Deep-Learning-Papers-Reading-Roadmap',
 'https://github.com/kmario23/deep-learning-drizzle',
 'https://github.com/kmario23/deep-learning-drizzle',
 'https://github.com/coqui-ai/TTS',
 'https://github.com/Kulbear/deep-learning-coursera',
 'https://github.com/d2l-ai/d2l-en',
 'https://github.com/baidu-research/DeepBench',
 'https://github.com/ChristosChristofidis/awesome-deep-learning',
 'https://github.com/david-gpu/srez']
```
q2 = "learning resources for computer vision"

```
L2 distance: [138.74569702148438, 169.27291870117188, 173.50767517089844, 176.0240936279297, 178.26512145996094, 179.61874389648438, 182.32862854003906, 183.37486267089844, 185.71066284179688, 185.7895965576172]

Readme IDs: [682, 1330, 420, 1138, 132, 1281, 415, 1232, 1187, 318]
['https://github.com/kenjihata/cs231a-notes',
 'https://github.com/ankonzoid/artificio',
 'https://github.com/llSourcell/Learn_Computer_Vision',
 'https://github.com/khanhnamle1994/computer-vision',
 'https://github.com/microsoft/muzic',
 'https://github.com/dformoso/machine-learning-mindmap',
 'https://github.com/aws-samples/aws-machine-learning-university-accelerated-cv',
 'https://github.com/jrobchin/Computer-Vision-Basics-with-Python-Keras-and-OpenCV',
 'https://github.com/dusty-nv/jetson-inference',
 'https://github.com/chainer/chainercv']
 ```
q3 = "infrastructure based on the asp.net core"

```
L2 distance: [202.02178955078125, 202.56175231933594, 203.72470092773438, 205.60671997070312, 212.73658752441406, 213.1620635986328, 216.88650512695312, 219.00022888183594, 220.93382263183594, 221.25343322753906]

Readme IDs: [475, 818, 772, 526, 251, 711, 1347, 1245, 88, 873]
['https://github.com/thingsboard/thingsboard',
 'https://github.com/bridgecrewio/checkov',
 'https://github.com/abpframework/abp',
 'https://github.com/aspnetboilerplate/aspnetboilerplate',
 'https://github.com/ReactiveX/RxSwift',
 'https://github.com/alibaba/x-deeplearning',
 'https://github.com/graphql/dataloader',
 'https://github.com/Jackpopc/aiLearnNotes',
 'https://github.com/hashicorp/terraform',
 'https://github.com/dotnet-architecture/eShopOnWeb']
```
q4 = "nlp chat bots"

```
L2 distance: [197.46856689453125, 198.1324005126953, 202.7813720703125, 208.62408447265625, 208.9859619140625, 211.4918670654297, 212.06109619140625, 213.52980041503906, 222.63330078125, 222.7926483154297]

Readme IDs: [946, 826, 608, 432, 860, 950, 1185, 1047, 145, 1382]
['https://github.com/BotLibre/BotLibre',
 'https://github.com/LeiWang1999/ZYNQ-NVDLA',
 'https://github.com/DevDungeon/Cathy',
 'https://github.com/chrisalbon/mlai',
 'https://github.com/RubensZimbres/Repo-2016',
 'https://github.com/yanshengjia/artificial-intelligence',
 'https://github.com/deepmipt/DeepPavlov',
 'https://github.com/andelf/PyAIML',
 'https://github.com/getsentry/sentry',
 'https://github.com/aidlearning/AidLearning-FrameWork']
 ```
q7 = "password manager"

 ```
L2 distance: [155.17620849609375, 194.96591186523438, 251.89572143554688, 255.93499755859375, 262.82598876953125, 263.5931701660156, 264.57269287109375, 267.79254150390625, 268.3603210449219, 271.2806701660156]

Readme IDs: [355, 1153, 1249, 1385, 570, 1191, 840, 402, 577, 565]
['https://github.com/marcwebbie/passpie',
 'https://github.com/keepassxreboot/keepassxc',
 'https://github.com/keycloak/keycloak',
 'https://github.com/houtianze/bypy',
 'https://github.com/jpillora/cloud-torrent',
 'https://github.com/owncloud/core',
 'https://github.com/kzlekk/HWSensors',
 'https://github.com/Gazler/githug',
 'https://github.com/facebookarchive/RakNet',
 'https://github.com/GoogleCloudPlatform/python-docs-samples']
 ```




